In [8]:
import cv2
from matplotlib import pyplot as plt

# Küresel değişkenler
drawing = False  # Fare basılı mı?
ix, iy = -1, -1  # Başlangıç koordinatları
boxes = []  # Etiketlenen kutuların listesi

# Fare olayları için geri çağırma fonksiyonu
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, boxes, image

    if event == cv2.EVENT_LBUTTONDOWN:  # Sol tıklama ile başlar
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:  # Fare hareket ederken çizim yapılır
        if drawing:
            img_copy = image.copy()
            cv2.rectangle(img_copy, (ix, iy), (x, y), (0, 255, 0), 2)
            cv2.imshow('Image', img_copy)

    elif event == cv2.EVENT_LBUTTONUP:  # Sol tıklama bittiğinde dikdörtgen çizilir
        drawing = False
        boxes.append((ix, iy, x, y))  # Dikdörtgenin başlangıç ve bitiş koordinatlarını kaydet
        cv2.rectangle(image, (ix, iy), (x, y), (0, 255, 0), 2)
        cv2.imshow('Image', image)

# Görseli yükle
image = cv2.imread('street.jpeg')  # Kendi resminizi buraya ekleyin

# Pencereyi oluştur ve fare olaylarını ayarla
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', draw_rectangle)

# Görseli göster ve işlem bitene kadar bekle
while True:
    cv2.imshow('Image', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' tuşuna basarak çıkış yapabilirsiniz
        break

# Etiketlenmiş dikdörtgenleri kaydet
cv2.destroyAllWindows()

# Etiketlenen kutuları ekrana yazdır
for i, box in enumerate(boxes):
    print(f"Araba {i + 1}: Başlangıç noktası: ({box[0]}, {box[1]}), Bitiş noktası: ({box[2]}, {box[3]})")


Araba 1: Başlangıç noktası: (363, 446), Bitiş noktası: (613, 622)
Araba 2: Başlangıç noktası: (647, 477), Bitiş noktası: (760, 564)


In [9]:
import json

# Etiketlenen bölgeleri bir JSON dosyasına kaydet
with open('annotations.json', 'w') as f:
    json.dump(boxes, f)


In [10]:
import cv2
import json

# Görseli yükle
image = cv2.imread('street.jpeg')

# JSON dosyasından etiketlenen bölgeleri yükle
with open('annotations.json', 'r') as f:
    boxes = json.load(f)

# Etiketlenen bölgeleri görselleştir
for box in boxes:
    cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

# Görseli göster
cv2.imshow('Annotated Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [12]:
import cv2
import numpy as np

# Küresel değişkenler
drawing = False  # Fare basılı mı?
ix, iy = -1, -1  # Başlangıç koordinatları
points = []  # Çokgenin köşe noktaları
polygons = []  # Çizilen çokgenlerin listesi

# Fare olayları için geri çağırma fonksiyonu
def draw_polygon(event, x, y, flags, param):
    global ix, iy, drawing, points, polygons, image

    if event == cv2.EVENT_LBUTTONDOWN:  # Sol tıklama ile başlar
        if not drawing:
            drawing = True
            points = [(x, y)]  # Yeni çokgen başlat
        else:
            points.append((x, y))  # Yeni nokta ekle
            # Geçici çokgen çizimi
            img_copy = image.copy()
            cv2.polylines(img_copy, [np.array(points)], isClosed=False, color=(0, 255, 0), thickness=2)
            cv2.imshow('Image', img_copy)

    elif event == cv2.EVENT_RBUTTONDOWN:  # Sağ tıklama ile çokgen tamamlanır
        if drawing:
            drawing = False
            polygons.append(points)  # Çokgeni kaydet
            # Tamamlanmış çokgeni çiz
            cv2.polylines(image, [np.array(points)], isClosed=True, color=(0, 255, 0), thickness=2)
            cv2.imshow('Image', image)

# Görseli yükle
image = cv2.imread('park.jpg')  # Kendi resminizi buraya ekleyin

# Pencereyi oluştur ve fare olaylarını ayarla
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', draw_polygon)

# Görseli göster ve işlem bitene kadar bekle
while True:
    cv2.imshow('Image', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' tuşuna basarak çıkış yapabilirsiniz
        break

# Etiketlenen çokgenleri aynı dosyaya kaydet
cv2.imwrite('street_polygons.jpeg', image)

# Etiketlenmiş görseli tekrar göster
image_with_polygons = cv2.imread('street_polygons.jpeg')
cv2.imshow('Annotated Image', image_with_polygons)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Etiketlenen çokgenleri ekrana yazdır
for i, polygon in enumerate(polygons):
    print(f"Çokgen {i + 1}: Noktalar: {polygon}")


Çokgen 1: Noktalar: [(42, 300)]
Çokgen 2: Noktalar: [(198, 216), (498, 202), (499, 174), (383, 162)]
Çokgen 3: Noktalar: [(28, 225), (26, 312), (181, 314), (216, 256), (175, 220)]


In [13]:
# metinsel verilerin anotasyonu 
#--------------------------------------

In [14]:
!pip install ipywidgets

In [19]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Metin verisini yükleme
def load_text_file(file_path):
    """Bir metin dosyasını yükler ve cümlelere böler."""
    with open(file_path, 'r') as file:
        content = file.read()
    return content.split('\n')  # Cümleleri satır bazında ayırır

# Anotasyon arayüzü
class TextAnnotator:
    def __init__(self, text_lines):
        self.text_lines = text_lines
        self.annotations = []
        self.index = 0
        
        # Etiketler
        self.labels = ['Kategori A', 'Kategori B', 'Kategori C']
        
        # Arayüz bileşenleri
        self.text_area = widgets.Textarea(
            value=self.text_lines[self.index],
            description='Cümle:',
            layout=widgets.Layout(width='80%')
        )
        
        self.label_selector = widgets.Dropdown(
            options=self.labels,
            description='Etiket:',
        )
        
        self.next_button = widgets.Button(description="Sonraki")
        self.save_button = widgets.Button(description="Kaydet")
        
        # Etkileşimler
        self.next_button.on_click(self.next_line)
        self.save_button.on_click(self.save_annotations)
        
        # Görüntüleme
        display(self.text_area, self.label_selector, self.next_button, self.save_button)
        
    def next_line(self, _):
        """Sonraki cümleye geçer."""
        if self.index < len(self.text_lines) - 1:
            self.annotations.append((self.text_lines[self.index], self.label_selector.value))
            self.index += 1
            self.text_area.value = self.text_lines[self.index]
        else:
            self.annotations.append((self.text_lines[self.index], self.label_selector.value))
            print("Tüm cümleler etiketlendi.")
            self.text_area.value = "Tamamlandı!"
            self.next_button.disabled = True

    def save_annotations(self, _):
        """Etiketlenmiş verileri bir dosyaya kaydeder."""
        df = pd.DataFrame(self.annotations, columns=['Cümle', 'Etiket'])
        df.to_csv('annotated_text.csv', index=False)
        print("Etiketlenmiş veriler kaydedildi: annotated_text.csv")

# Metin dosyasının yolunu belirtin
input_file = 'yapay_zeka.txt'  # Kendi metin dosyanızın yolunu buraya ekleyin

# Metin verilerini yükle
text_lines = load_text_file(input_file)

# Anotasyon arayüzünü başlat
annotator = TextAnnotator(text_lines)


Textarea(value='â€˜Yapay Zekaâ€™ kavramÄ± ilk duyuÅŸta ister akademisyen, Ã¶ÄŸretmen, Ã¶ÄŸrenci olsun ister', …

Dropdown(description='Etiket:', options=('Kategori A', 'Kategori B', 'Kategori C'), value='Kategori A')

Button(description='Sonraki', style=ButtonStyle())

Button(description='Kaydet', style=ButtonStyle())

Tüm cümleler etiketlendi.
